In [ ]:
#2017/4/15 完成訓練模組
#2017/4/21 加入RUC曲線
#考慮自由中國非文藝類，投稿數量前10名的作者
#實驗流程：獲得內文 -> 文本特徵向量建立 -> SMOTE平衡文本量 
# -> GridSearchCV找尋最佳參數並交叉驗證 -> 建立預測報表/點陣圖/預測模型
#特徵採用詞性組合、情態詞+V組合、否定&程度組合
#一般常用的特徵有高頻詞、bigram&trigram、標點符號、豐富度
import codecs
import os
from collections import OrderedDict,defaultdict,Counter
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,accuracy_score
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from matplotlib.font_manager import FontProperties
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix
from imblearn.under_sampling import OneSidedSelection 

condicate_author = ['雷震','殷海光','夏道平','傅正','龍平甫','蔣勻田','朱伴耘','胡適','羅鴻詔']

condicate_author_path = "D:\\課業相關\\論文資料\\SVM\\condicate_author\\" #候選作者文本
normal_feature_path = "D:\\課業相關\\論文資料\\SVM\\language_feature\\" #常用特徵
paper_feature_path = "D:\\課業相關\論文資料\\SVM\\language_feature\\三版\\" #論文提出特徵
unknow_author_path = "D:\\課業相關\\論文資料\\SVM\\unknow_author\\" #未知作者文本

print ("library and path OK")

In [ ]:
#建立作者索引，提取各作者文章內容及索引
author_index = defaultdict(int)
for index,name in enumerate(condicate_author): #建立作者索引
    author_index[name] = index

author_index = OrderedDict(sorted(author_index.items(), key=lambda t: t[1])) #作者索引排序(依文本數量高到低)

for name,index in author_index.items():
    print (name+':'+str(index),end=' ')
print ()
    
content_list = [] #所有作者文本內容(未處理)
article_label = [] #各文本label，也就是作者索引

a_c = defaultdict(int)

for file in os.listdir(condicate_author_path):
    with codecs.open(condicate_author_path+file,'rb','utf8') as f:
        
        if file.split('_')[0] not in condicate_author:
            continue
        
        '''if a_c[file.split('_')[0]] > 19:
            continue'''
        
        title = f.readline()
        content = f.readline().strip()
        
        '''if len(content.split()) <= 2000:
            continue'''
        
        content_list.append(content)
        
        article_label.append(author_index[file.split('_')[0]])
        
        '''if len(content.split()) <= 2000:
            content_list.append(content)
            article_label.append(author_index[file.split('_')[0]])'''
        
        #a_c[file.split('_')[0]] += 1
        
print ('共有'+str(len(content_list))+'篇文章')
print (Counter(article_label))

In [ ]:
#建立特徵
feature = [] #特徵
feature_file = '自由中國前100個N+N.csv' 
select_path = paper_feature_path
with codecs.open(select_path+feature_file,'rb','utf8') as f: #抓取基準特徵
    for i in f.readlines():
        if i.strip() != '':
            feature.append(i.strip().split(',')[0])
            
feature = feature[:50]

#單以作者間為母體
'''feature_select = defaultdict(int)

def format_check(word):
    if '(' in word and 'CATEGORY' not in word and word != '' and word[0] != '(' and word.split('(')[1] != '':
        return True
    return False
        
def pos_check(word1,word2,pos1,pos2):
    if word1.split('(')[1][0] == pos1 and word2.split('(')[1][0] == pos2:
        return True
    return False
        
for file in content_list:
    content = file.split()
    for i in range(len(content)-1):
        if format_check(content[i]) and format_check(content[i+1]):
            if content[i].split('(')[1][:2] == 'VH' and content[i+1].split('(')[1][0] == 'N' \
                    and content[i+1].split('(')[1].split(')')[0] != 'Neu':
                feature_select[content[i]+content[i+1]] += 1

feature_select = sorted(feature_select.items(), key=lambda d:d[1], reverse = True)

for i,(f,n) in enumerate(feature_select):
    if i == 100:
        break
    feature.append(f)'''
    
print (len(feature))   
print (feature)

In [ ]:
#將文章轉換為特徵向量，並平衡作者文本數量(以最多為基準)
def line_vec(line): #將文章轉換為特徵向量並回傳
    temp_feature = defaultdict(int)
    
    #line = [line[i].split('(')[0] for i in range(len(line))] #詞頻
    #line = [line[i].split('(')[0]+line[i+1].split('(')[0] for i in range(len(line)-1)] #bigram
    #line = [line[i].split('(')[0]+line[i+1].split('(')[0]+line[i+2].split('(')[0] for i in range(len(line)-2)] #trigram
    line = [line[i]+line[i+1] for i in range(len(line)-1)] #詞性組合
    #line = [line[i]+line[i+1] for i in range(len(line)-1)] + [line[i]+line[i+1]+line[i+2] for i in range(len(line)-2)] #2~3詞性組合
    
    for i in line:
        if i in feature:
            temp_feature[i] += 1
    
    return temp_feature

aa_feature = np.zeros((len(content_list),len(feature)),np.float64)
muti_author = defaultdict(list) #將向量分類

for index,element in enumerate(content_list): #依序將文章轉換為特徵向量
    line = element.split()
    temp_feature = line_vec(line)
    
    for i,j in enumerate(feature):
        aa_feature[index, i] = round(temp_feature[j] * 1000000 / len(line)) #取相對頻率
        
    muti_author[article_label[index]].append(list(aa_feature[index]))
    
'''for i,e in muti_author.items():
    for j in e:
        print (i,max(j),Counter(j))
    print ()'''

for index in range(1,len(muti_author)): #將分類後的向量內容數量平均，也就是文本數量一致

    min_target = -1
    min_num = 1000
    
    for i,e in muti_author.items():
        if len(e) <= min_num:
            min_num = len(e)
            min_target = i
            
    if len(muti_author[0]) == min_num: #取到文本數量與最大等數值為止，預設最大是第0個
        break
    
    X_t = muti_author[0]+muti_author[min_target]
    y_t = [0]*len(muti_author[0])+[min_target]*len(muti_author[min_target])
    
    X_t, y_t = SMOTE(random_state=0).fit_sample(X_t,y_t) #SMOTE
    muti_author[min_target] = X_t[len(muti_author[0]):]


'''for a_i in range (len(muti_author)):
    for index in range(a_i+1,len(muti_author)):

        X_t = muti_author[a_i]+muti_author[index]
        y_t = [a_i]*len(muti_author[a_i])+[index]*len(muti_author[index])

        X_t, y_t = OneSidedSelection(random_state=0).fit_sample(X_t,y_t) #OSS
        
        if Counter(y_t)[a_i] <= 40 and Counter(y_t)[index] <= 40:
            continue
        
        muti_author[index] = [X_t[j] for j in [i for i,e in enumerate(y_t) if e == index]]
        muti_author[a_i] = [X_t[j] for j in [i for i,e in enumerate(y_t) if e == a_i]]
        
max_target = 0
max_num = -1
for k,v in muti_author.items():
    if len(v) > max_num:
        max_num = len(v)
        max_target = k
        
for index in range(1,len(muti_author)): #將分類後的向量內容數量平均，也就是文本數量一致

    min_target = -1
    min_num = 1000
    
    for i,e in muti_author.items():
        if len(e) <= min_num:
            min_num = len(e)
            min_target = i
            
    if len(muti_author[max_target]) == min_num: #取到文本數量與最大等數值為止，預設最大是第0個
        break
    
    X_t = muti_author[max_target]+muti_author[min_target]
    y_t = [max_target]*len(muti_author[max_target])+[min_target]*len(muti_author[min_target])
    
    X_t, y_t = SMOTE(random_state=0).fit_sample(X_t,y_t) #SMOTE
    muti_author[min_target] = X_t[len(muti_author[max_target]):]'''

    
X = [] #處理後文本向量
y = [] #處理後文本label

#X,y = OneSidedSelection(random_state=0).fit_sample(aa_feature,article_label)

for k,v in muti_author.items():
    for l in v:
        X.append(l)
    y += [k]*len(v)
    
X = np.array(X)
y = np.array(y)

#X,y = OneSidedSelection(random_state=0).fit_sample(X,y)

print ('X:',X.shape)
print ('y:',y.shape)
print ('資料準備已完成')
print (Counter(list(y)))

zero_count = []
for i in range(len(feature)):
    zero_count.append(Counter(X[:,i].tolist())[0]) #找出特徵中0的出現次數
print (zero_count)

In [ ]:
def sparsity_ratio(X):
    return 1.0 - np.count_nonzero(X) / float(X.shape[0] * X.shape[1])
print("input sparsity ratio:", sparsity_ratio(X))

In [ ]:
from sklearn import preprocessing 

X = preprocessing.scale(X)
print (X[:10])

In [ ]:
#建立預測模型
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42) #分割訓練與測試資料

print (feature_file)
print ()

start = time.time()

#MultinomialNB
'''from sklearn.naive_bayes import MultinomialNB

param_grid = {
    'fit_prior': [True,False]
}

clf = GridSearchCV(MultinomialNB(), n_jobs=-1, param_grid=param_grid, cv=10)
clf.fit(X_train,y_train)'''


#決策樹
'''param_grid = {
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

tclf = DecisionTreeClassifier(max_features= 'sqrt')
clf = GridSearchCV(tclf, param_grid=param_grid, cv=10)
clf.fit(X_train, y_train)'''


#SVC
'''tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], #SVM
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}] #候選參數

clf = GridSearchCV(svm.SVC(C=1,probability=True), tuned_parameters, n_jobs=-1, cv=10) #使用交叉驗證並找出最佳參數
clf.fit(X_train, y_train)'''

#LinearSVC
'''param_grid = {
    'C': [1, 10, 100, 1000],
    'dual': [True,False],
    'loss': ["squared_hinge","hinge"],
    'penalty': ["l1","l2"]
}

clf = GridSearchCV(svm.LinearSVC(class_weight='balanced'), param_grid=param_grid, 
                   n_jobs=-1, cv=10, error_score=0.0) #使用交叉驗證並找出最佳參數
clf.fit(X_train, y_train)'''

#隨機森林樹
param_grid = { 
    'n_estimators': [200, 700], #1000和700結果差不多
    'max_features': ['auto', 'sqrt', 'log2']
}

rfc = RandomForestClassifier(n_jobs=-1, oob_score = True, class_weight = 'balanced')
clf = GridSearchCV(rfc, param_grid=param_grid, cv=10, error_score=0.0) #entropy沒差別
clf.fit(X_train, y_train)

end = time.time()
print ('耗費時間：',end-start,'秒')
print ()

print("Best parameters set found on development set:")
print()
print(clf.best_params_) #印出最佳參數
print()
print ('最佳預測精準度：',clf.best_score_)
print ()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.") 
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred)) #顯示報表(pred可能和predict_proba不一致，這邊假設為一致)
print (accuracy_score(y_true, y_pred))
print()

In [ ]:
from sklearn.metrics import hamming_loss
print (hamming_loss(y_true, y_pred))

In [ ]:
#建立困惑矩陣
import itertools
ZF1 = FontProperties(fname='C:\Windows\Fonts\kaiu.ttf', size=14)
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontproperties=ZF1)
    plt.yticks(tick_marks, classes, fontproperties=ZF1)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        #print('Confusion matrix, without normalization')
        pass

    #print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, round(cm[i, j],2),
                 horizontalalignment="center", verticalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(8, 6))
plot_confusion_matrix(cnf_matrix, classes=condicate_author,
                      title='Confusion matrix, without normalization')

plt.savefig('C:\\Users\\user\\Desktop\\0001.png', bbox_inches="tight")

# Plot normalized confusion matrix
plt.figure(figsize=(8, 6))
plot_confusion_matrix(cnf_matrix, classes=condicate_author, normalize=True,
                      title='Normalized confusion matrix')

plt.savefig('C:\\Users\\user\\Desktop\\0002.png', bbox_inches="tight")

plt.show()

In [ ]:
#比較個演算法
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0) #分割訓練與測試資料

print (feature_file)

gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

def NB_pre(pre):
    clf = GridSearchCV(pre, n_jobs=-1, param_grid={}, cv=10)
    clf.fit(X_train,y_train)
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    return clf.best_score_
    

#測試分類
print('Gaussian 分類:', NB_pre(gnb))
print ()
print('Multinomial 分類:', NB_pre(mnb))
print ()
print('Bernoulli 分類:', NB_pre(bnb))
print ()

clf = svm.LinearSVC()
clf.fit(X_train,y_train)
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))

'''param_grid = {
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

tclf = DecisionTreeClassifier(max_features= 'sqrt')
clf = GridSearchCV(tclf, param_grid=param_grid, cv=10)
clf.fit(X_train, y_train)
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print('DecisionTree 分類:', clf.best_score_)
print ()

param_grid = { 
    'n_estimators': [200, 700], #1000和700結果差不多
    'max_features': ['auto', 'sqrt', 'log2']
}

rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True)
clf = GridSearchCV(rfc, n_jobs=-1, param_grid=param_grid, cv=10)
clf.fit(X_train, y_train)
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print('RandomForest 分類:', clf.best_score_)
print ()'''

In [ ]:
#重要特徵選取
from sklearn.feature_selection import SelectFromModel

model = SelectFromModel(clf.best_estimator_, prefit=True)
X_new = model.transform(X)
print (X_new.shape)
#X = X_new

'''zero_count = []
for i in range(X_new.shape[1]):
    zero_count.append(Counter(X_new[:,i].tolist())[0]) #找出特徵中0的出現次數
print (zero_count)'''

print ([feature[i] for i in range(len(feature)) if model.get_support()[i]])

In [ ]:
#未知作者預測
print (' '.join([str(i)+'.'+e for i,e in enumerate(condicate_author)])) #印出作者
print ()
 
print ('最佳預測精準度：',clf.best_score_)
print ()

for file in os.listdir(unknow_author_path):
    pred_y = np.zeros((1,len(feature)),np.float64)
    with codecs.open(unknow_author_path+file,'rb','utf8') as f:
        head = f.readline().strip()
        content = f.readline().strip().split()

        temp_feature = line_vec(content)

        for i,j in enumerate(feature):
            pred_y[0][i] = round(temp_feature[j] * 1000000 / len(content),3)

    print (head)
    print ('預測作者：'+str(clf.predict(pred_y)[0])+'.'+condicate_author[clf.predict(pred_y)[0]])
    print ([round(i,3) for i in clf.predict_proba(pred_y)[0].tolist()])
    print ()

In [ ]:
#畫出點陣圖
X_reduced = PCA(n_components=2).fit_transform(X) #使用PCA降維
colors = ['white', 'red', 'lime', 'cyan', 'orange', 'black', 'blue', 'purple', 'yellow']
for i in range(len(colors)):
    if i in [4,5,6,8]:
        continue
    x_p = X_reduced[:, 0][y == i]
    y_p = X_reduced[:, 1][y == i]
    plt.scatter(x_p, y_p, c=colors[i])
name = [condicate_author[i] for i in [0,1,2,3,7]]
plt.legend(name, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., \
           prop = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=14) ) #顯示中文
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title("PCA Scatter Plot")
plt.show()

In [ ]:
#預測模型儲存
from sklearn.externals import joblib
#joblib.dump(clf, 'C:\\Users\\user\\Desktop\\預測資料\\random forest\\自程.pkl') 
clf = joblib.load('C:\\Users\\user\\Desktop\\預測資料\\random forest\\自D+V.pkl') 
print (clf.best_score_)

In [ ]:
#印出特徵重要性
print (len(clf.best_estimator_.feature_importances_))
feature_score = list(clf.best_estimator_.feature_importances_)
#print (feature_score)
#print (feature_score.index(max(feature_score)))
feature_value,feature_index = zip(*sorted(zip(feature_score,[i for i,e in enumerate(feature_score)]),reverse=True)) #排序
print (feature_index[:10])
print (feature_value[:10])
print ([feature[i] for i in feature_index[:10]])
#print (feature_value[:10])
print ([i for i in zip(y,X[:,feature_index[0]])][:10])
print ([i for i in zip(article_label,aa_feature[:,feature_index[0]])][:5])
s = aa_feature[:,[i for i in feature_index[:10]]]
print (s.shape)
#print (aa_feature[:,feature_index[0]])
'''with codecs.open('C:\\Users\\user\\Desktop\\前10名特徵向量.csv','wb','utf8') as g:
    for i,e in enumerate(list(s)):
        g.write(str(article_label[i])+','+','.join(list(map(str,e)))+'\r\n')'''

In [ ]:
#AUC計算(待修正)
#y_true2, y_pred2 = y_test, clf.predict_proba(X_test)
#y_true2, y_pred2 = y_test, clf.predict(X_test)
y_true2, y_pred2 = y_test, clf.decision_function(X_test)

colors = ['green', 'red', 'lime', 'cyan', 'orange', 'black', 'blue', 'purple', 'yellow']
lw = 2
tag = []

mean = 0.0

for i in range(len(condicate_author)):
    fpr, tpr,thresholds = roc_curve(y_true2, y_pred2[:,i], pos_label=i)
    #fpr, tpr,thresholds = roc_curve([1 if j == i else 0 for j in y_true2], [1 if j == i else 0 for j in y_pred2])
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, color=colors[i], lw=lw)
    tag.append('%s ROC curve (area = %0.2f)' % (condicate_author[i],roc_auc))  
    mean += roc_auc
    
print (round(mean/len(condicate_author),2))
    
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.legend(tag, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., \
           prop = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=14)) #顯示中文
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('All author\'s ROC curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#印出決策樹
#graphviz用conda安裝後要建立環境變數
from sklearn import tree
import pydotplus 
from IPython.display import Image 

dot_data = tree.export_graphviz(clf.best_estimator_[0], out_file=None,
                                filled=True,feature_names=feature,class_names=condicate_author,
                                 max_depth=None,proportion=True,rounded=True,special_characters=True) 
dot_data = dot_data.replace('helvetica','kaiu') #字型調換
graph = pydotplus.graph_from_dot_data(dot_data)
#graph.write_pdf("C:\\Users\\user\\Desktop\\graph.pdf") 
#Image(graph.create_png())
graph.write_png("C:\\Users\\user\\Desktop\\rtree.png")
#[str(i) for i,e in enumerate(condicate_author)]

In [ ]:
#輸出決策樹規則(未清理)
tree_content = []
def get_code(tree, feature_names, target_names,
             spacer_base="    "):
    """Produce psuedo-code for decision tree.

    Args
    ----
    tree -- scikit-leant DescisionTree.
    feature_names -- list of feature names.
    target_names -- list of target (class) names.
    spacer_base -- used for spacing code (default: "    ").

    Notes
    -----
    based on http://stackoverflow.com/a/30104792.
    """
    left      = tree.tree_.children_left
    right     = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features  = [feature_names[i] for i in tree.tree_.feature]
    value = tree.tree_.value

    def recurse(left, right, threshold, features, node, depth):
        spacer = spacer_base * depth
        if (threshold[node] != -2):
            '''print(spacer + "if ( " + features[node] + " <= " + \
                  str(threshold[node]) + " ) {")'''
            tree_content.append(spacer + "if ( " + features[node] + " <= " + \
                  str(threshold[node]) + " ) {")
            if left[node] != -1:
                    recurse(left, right, threshold, features,
                            left[node], depth+1)
            #print(spacer + "}\n" + spacer +"else {")
            tree_content.append(spacer + "}\n" + spacer +"else {")
            if right[node] != -1:
                    recurse(left, right, threshold, features,
                            right[node], depth+1)
            #print(spacer + "}")
            tree_content.append(spacer + "}")
        else:
            target = value[node]
            for i, v in zip(np.nonzero(target)[1],
                            target[np.nonzero(target)]):
                target_name = target_names[i]
                target_count = int(v)
                '''print(spacer + "return " + str(target_name) + \
                      " ( " + str(target_count) + " examples )")'''
                tree_content.append(spacer + "return " + str(target_name) + \
                      " ( " + str(target_count) + " examples )")

    recurse(left, right, threshold, features, 0, 0)
    
#print (clf.best_estimator_[0].tree_.children_left)
get_code(clf.best_estimator_[0], feature, condicate_author)
with codecs.open('C:\\Users\\user\\Desktop\\rule.txt','wb','utf8') as g:
    for i in tree_content:
        g.write(i+'\r\n')
print ('END')

In [ ]:
#輸出各作者決策樹規則
tree_path = []
tree_all_path = []

author_path = defaultdict(dict)
for i in condicate_author:
    author_path[i] = {}

def get_code(tree, feature_names, target_names,
             spacer_base="    "):
    """Produce psuedo-code for decision tree.

    Args
    ----
    tree -- scikit-leant DescisionTree.
    feature_names -- list of feature names.
    target_names -- list of target (class) names.
    spacer_base -- used for spacing code (default: "    ").

    Notes
    -----
    based on http://stackoverflow.com/a/30104792.
    """
    left      = tree.tree_.children_left
    right     = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features  = [feature_names[i] for i in tree.tree_.feature]
    value = tree.tree_.value

    def recurse(left, right, threshold, features, node, depth):
        spacer = spacer_base * depth
        if (threshold[node] != -2):
            tree_path.append(features[node]+' <= '+str(threshold[node]))
            if left[node] != -1:
                recurse(left, right, threshold, features,
                        left[node], depth+1)
            tree_path.pop()
            tree_path.append(features[node]+' > '+str(threshold[node]))
            if right[node] != -1:
                recurse(left, right, threshold, features,
                        right[node], depth+1)
            tree_path.pop()
        else:
            target = value[node]
            for i, v in zip(np.nonzero(target)[1],target[np.nonzero(target)]):
                target_name = target_names[i]
                target_count = int(v)
                temp = ' and '.join(tree_path)+':'+str(target_name)+" (" + str(target_count) + " examples)"
                #tree_all_path.append(' and '.join(tree_path)+':'+str(target_name)+" (" + str(target_count) + " examples)")
                if target_count not in author_path[target_name]:
                    author_path[target_name][target_count] = [temp]
                else:
                    author_path[target_name][target_count].append(temp)

    recurse(left, right, threshold, features, 0, 0)
    
get_code(clf.best_estimator_[0], feature, condicate_author)
'''with codecs.open('C:\\Users\\user\\Desktop\\all_path.txt','wb','utf8') as g:
    for i in tree_all_path:
        g.write(i+'\r\n')'''

for name in author_path: 
    l = sorted([i for i in author_path[name]],reverse=True)
    with codecs.open('C:\\Users\\user\\Desktop\\author_path\\'+name+'.txt','wb','utf8') as g:
        for i in l:
            g.write(str(i)+'\r\n')
            for e in author_path[name][i]:
                g.write(e+'\r\n')
            g.write('\r\n')
            
print ('END')

In [ ]:
#針對mutlclass的ROC圖(嘗試中)
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

# Binarize the output
y = label_binarize(y, classes=[i for i in range(len(condicate_author))])
n_classes = y.shape[1]

# Add noisy features to make the problem harder
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
#X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2,
                                                    random_state=0)

# Learn to predict each class against the other
classifier = OneVsRestClassifier(clf)
y_score = classifier.fit(X_train, y_train).decision_function(X_test)
#y_score = classifier.fit(X_train, y_train).predict_proba(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr[0], tpr[0], label='ROC curve (area = %0.2f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

tag = []
# Plot ROC curve
plt.figure()
'''plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]))'''
plt.plot(fpr["micro"], tpr["micro"])
tag.append('micro-average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]))
for i in range(n_classes):
    '''plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                   ''.format(i, roc_auc[i]))'''
    plt.plot(fpr[i], tpr[i])
    tag.append('%s ROC curve (area = %0.2f)' % (condicate_author[i],roc_auc[i]))

plt.legend(tag, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., \
           prop = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=14)) #顯示中文
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()